In [1]:
#models & metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search


#modules
import pandas as pd
import numpy as np
import re

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load data
train_df = pd.read_csv(r'/Users/brodyuehara/Desktop/Titanic Machine Learning Project/train.csv')
test_df = pd.read_csv(r'/Users/brodyuehara/Desktop/Titanic Machine Learning Project/test.csv')
TestId = test_df['PassengerId']

In [3]:
print("Dimensions of train: {}".format(train_df.shape))
print("Dimensions of test: {}".format(test_df.shape))

Dimensions of train: (891, 12)
Dimensions of test: (418, 11)


In [4]:
#Remove outliers
#train_df = train_df.drop(Outliers_to_drop, axis=0).reset_index(drop=True)

In [5]:
#Join train and test sets
#Join to obtain the same number of features during categorical concerison
train_idx = len(train_df)
df = pd.concat([train_df,test_df], axis=0).reset_index(drop=True)

# Missing Values

In [6]:
#Fill empty and NA values with NaN
df = df.fillna(np.nan)

#check for Null values
df.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [7]:
# Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived 
#g = sns.heatmap(train_df[["Survived","SibSp","Parch","Age","Fare"]].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")

In [8]:
#new feature to extract title names from the Name column
df["Title"] = df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())

#show count of titles
print("There are {} unique titles.".format(df.Title.nunique()))

#show unique titles
print("\n", df.Title.unique())

There are 18 unique titles.

 ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


In [9]:
#normalize the titles
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

#map normolized titles to the current titles
df.Title = df.Title.map(normalized_titles)

#view value counts for the normalized titles
print(df.Title.value_counts())

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64


In [10]:
#apply the grouped median value on null ages
df['Age']=df.groupby(['Sex', 'Title', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))

#view changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 133.0+ KB


In [11]:
#extract cabin deck 
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [df]

for dataset in data: 
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
    
#drop cabin feature
df = df.drop(['Cabin'], axis=1)

In [12]:
#fill in two missing females as S according to google
df['Embarked'] = df['Embarked'].fillna('S')

In [13]:
#fill NaN with median fare
med_fare = df.groupby(['Pclass', 'Parch', "SibSp"]).Fare.median()[3][0][0]

#fill in missing values in fare with the median fare of 3rd class alone passenger
df['Fare'] = df['Fare'].fillna(med_fare)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
Age            1309 non-null float64
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
Title          1309 non-null object
Deck           1309 non-null int64
dtypes: float64(3), int64(5), object(5)
memory usage: 133.0+ KB


# New Features

In [15]:
#size of families (inluding the passenger)
df['FamilySize'] = df.Parch + df.SibSp + 1

#new features for familysize
df['Single'] = df['FamilySize'].map(lambda x: 1 if x == 1 else 0)
df['SmallFamily'] = df['FamilySize'].map(lambda x: 1 if x == 2 else 0)
df['MediumFamily'] = df['FamilySize'].map(lambda x: 1 if 3 <= x <= 4 else 0)
df['LargeFamily'] = df['FamilySize'].map(lambda x: 1 if x >= 5 else 0)

In [16]:
#Fare per person
df['Fare_Per_Person'] = df.Fare/(df.SibSp + df.Parch + 1)
df['Fare_Per_Person'] = df.Fare_Per_Person.astype(int)

In [17]:
#Dummy variables
df = pd.get_dummies(df, columns = ['Title'])
df = pd.get_dummies(df, columns = ['Embarked'], prefix='Em')
df = pd.get_dummies(df, columns = ["Ticket"], prefix="T")

In [18]:
#convert sex
genders = {"male": 0, "female": 1}
data = [df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [ ]:
#Create categorical values for Pclass
df['Pclass'] = df['Pclass'].astype('category')
df = pd.get_dummies(df, columns = ['Pclass'],prefix='Pc')

In [ ]:
#Drop useless variables 
df.drop(labels = ['PassengerId'], axis = 1, inplace = True)
df.drop(labels = ['Name'], axis = 1, inplace = True)

# Modeling

In [ ]:
train = df[:train_idx]
test = df[train_idx:]

In [ ]:
#Separate train features and label 

train["Survived"] = train["Survived"].astype(int)

y = train["Survived"]

X = train.drop(labels = ["Survived"],axis = 1)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X,y, random_state=0)

# XGBoost

In [ ]:
model = xgb.XGBClassifier(booster='gbtree', silent=0, seed=0, base_score=0.5, subsample=0.75)
parameters = {'n_estimators':[240,280,320],
            'max_depth':[10,11,12],
            'gamma':[0,1,2,3],
            'max_delta_step':[0,1,2],
            'min_child_weight':[1,2,3], 
            'colsample_bytree':[0.55,0.6,0.65],
            'learning_rate':[0.1,0.2,0.3]
            }
model = GridSearchCV(model, parameters)
model.fit(train_X,train_y)
print('Best parameters founded : {}'.format(model.best_params_))

In [ ]:
model = xgb.XGBClassifier(booster='gbtree', silent=1, seed=0, base_score=0.5, subsample=0.75)
grid ={'max_delta_step': 0, 'max_depth': 10, 'min_child_weight': 2, 'n_estimators': 280, 'colsample_bytree': 0.65, 'gamma': 2}
model.set_params(**grid)
model.fit(train_X,train_y)
kfold = KFold(n_splits=10, random_state=7)
results = cross_val_score(model, np.array(val_X), np.array(val_y), cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
plot_importance(model)
plt.show()

In [ ]:
#predictions = model.predict(val_X)
print("Mean Absolute Error: "+ str(mean_absolute_error(predictions, val_y)))

In [ ]:
#submission_preds = pd.Series(model.predict(test), name = 'Survived')
accuracy = accuracy_score(val_y, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#df with predicitons
results = pd.concat([TestId, submission_preds], axis=1)
results.head(20)

In [ ]:
#save to csv
results.to_csv(r'C:\Users\buehara\Titanic\Titanic_kaggle.csv', index=False)

In [ ]:
#getting the probabilities of predictions
y_scores = model.predict_proba(val_X)
y_scores = y_scores[:,1]

precision, recall, threshold = precision_recall_curve(val_y, y_scores)

def plot_precision_and_recall(precision, recall, threshold):
    plt.plot(threshold, precision[:-1], "r-", label="precision", linewidth=2)
    plt.plot(threshold, recall[:-1], "b", label="recall", linewidth=2)
    plt.xlabel("threshold", fontsize=15)
    plt.legend(loc="upper right", fontsize=15)
    plt.ylim([0, 1])
    
plt.figure(figsize=(14, 7))
plot_precision_and_recall(precision, recall, threshold)
plt.show()

In [ ]:
#compute true positive rate and false positive rate
false_positive_rate, true_positive_rate, thresholds = roc_curve(val_y, y_scores)

#plotting them against each other
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'r', linewidth=2)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)
    
plt.figure(figsize=(14, 7))
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
#ROC AUC Score, Score represents the percentage chance theat model will be able to distinguish 
#between positive class and negative class. 
r_a_score = roc_auc_score(val_y, y_scores)
print("ROC-AUC-SCORE:", r_a_score)

In [ ]:
features = pd.DataFrame()
features['feature'] = train_X.columns
features['importance'] = model.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)